## Import libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [2]:
# import pandas as pd
# data = pd.read_csv('book-war-and-peace.txt', header = None, sep=" ", error_bad_lines=False);

In [2]:
txt = open('book-war-and-peace.txt', "r")
# print(txt.readline())
# print(txt.read(10))

In [3]:
book_chars = []

In [4]:
for line in txt:
    for char in line:
        book_chars.append(char)

In [5]:
txt.close()

In [6]:
# book_chars[:10]
# book_chars = book_chars[:100000]

In [7]:
s=set(book_chars)
print('Elements:',s)
print('Set size:', len(s))

Elements: {'n', ' ', ';', 'X', '9', 'L', 'B', '5', 'J', 'o', '1', 'Q', 't', 'm', '!', 'c', 'u', 'j', '.', 'i', 'E', 'g', '3', 'M', 'H', 'G', '4', 'P', 'Z', 's', 'e', '6', 'W', 'k', 'D', 'w', 'O', '=', 'q', 'à', 'S', 'V', 'T', 'F', '0', ')', 'ä', '-', 'K', '7', 'z', 'A', 'f', '\n', ',', 'b', 'x', ':', 'r', '?', 'v', '/', '8', 'C', 'p', 'd', 'U', 'l', 'I', 'é', 'R', 'h', 'ê', '2', 'y', 'N', '"', 'a', '(', '*', "'", 'Y'}
Set size: 82


In [8]:
book_dict = {}

In [9]:
i = 0
for char in s:
    book_dict[char] = i
    i += 1

In [10]:
len(book_chars)

3202303

In [11]:
book_dict["a"]

77

## One-hot encoding

In [12]:
book_chars = np.array(book_chars)
dict_size = len(s)
seq_len = 1
batch_size = 25

In [13]:
def one_hot_encode(sequence, dict_size, 
                   seq_len, batch_size,
                      num_batches, book_dict):
    # Creating a multi-dimensional array of zeros with the 
    # desired output shape
    features = np.zeros((batch_size, seq_len, dict_size), 
                        dtype=np.float32)

    # Replacing the 0 at the relevant character index 
    # with a 1 to represent that character
    for i in range(batch_size):
        for u in range(seq_len):
            char_ix = (num_batches - 1) * batch_size + i
            features[i, u, book_dict[sequence[char_ix]]] = 1
    return features

In [14]:
def target_encode(sequence, batch_size, char_dict, num_batches):
    batch_list = []
    for x in range(batch_size):
        char_idx = (num_batches - 1) * batch_size + x
        char_val = char_dict[sequence[char_idx]]
        batch_list.append(char_val)
    return batch_list

In [15]:
features = one_hot_encode(book_chars, dict_size, seq_len, batch_size, 2, book_dict)
print(features[-1])

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


## Test Train Split using 70-30

In [16]:
train_char_len = (int) (0.7 * len(book_chars))
test_char_len = len(book_chars) - train_char_len
print(len(book_chars), train_char_len, test_char_len)

3202303 2241612 960691


In [17]:
train_batches = (int) (train_char_len / batch_size)
test_batches = (int) (test_char_len / batch_size)
print(train_batches, test_batches,test_batches + train_batches )

89664 38427 128091


In [18]:
X_train = []
for i in range(train_batches):
    minibatch = one_hot_encode(book_chars, dict_size, seq_len, batch_size, i, book_dict)
    X_train.append(torch.Tensor(minibatch))

In [19]:
Y_train = []
for i in range(train_batches):
    Y_train.append(torch.LongTensor(target_encode(book_chars[1:], batch_size, book_dict, i)))

In [20]:
print(len(X_train))
print(X_train[-1][0])
print(len(Y_train))
print(Y_train[0])

89664
tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
89664
tensor([19, 15, 71,  1, 35, 30,  1, 77, 58, 30,  1,  0,  9, 12, 53, 15,  9,  0,
        29, 15, 19,  9, 16, 29, 18])


In [21]:
X_test = []
for i in range(train_batches, train_batches + test_batches):
    minibatch = one_hot_encode(book_chars, dict_size, seq_len, batch_size, i, book_dict)
    X_test.append(torch.Tensor(minibatch))

In [22]:
Y_test = []
for i in range(train_batches, train_batches + test_batches):
    Y_test.append(torch.LongTensor(target_encode(book_chars[1:], batch_size, book_dict, i)))

In [23]:
print(len(X_test))
print(X_test[-1][0])
print(len(Y_test))
print(Y_test[1])

38427
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
38427
tensor([71, 30,  1, 52, 16, 58,  0, 19, 12, 16, 58, 30,  1, 35, 77, 29, 53, 55,
        30, 19,  0, 21,  1, 15, 77])


In [24]:
X_train = X_train[:40000]
Y_train = Y_train[:40000]
X_test = X_test[:1200]
Y_test = Y_test[:1200]

In [25]:
def convert_book():
    char_vals = []
    for c in book_chars:
        char_vals.append(book_dict[c])
    return char_vals

In [26]:
%time vals = convert_book()
print(len(vals))

CPU times: user 900 ms, sys: 17.8 ms, total: 918 ms
Wall time: 919 ms
3202303


## Model definition

In [72]:
class RNN(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim, no_layers = 1):
        super(RNN, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.no_layers = no_layers
        
        self.rnn_layer = nn.RNN(self.input_dim, 
                        self.hidden_dim, self.no_layers,
                                batch_first = True,
                                nonlinearity='tanh')
        self.linear_out = nn.Linear(self.hidden_dim, self.output_dim)
        self.softmax = nn.LogSoftmax(dim = 1)
        
    def forward(self, x):
        batch_size = x.size(0)
        hidden = torch.zeros(self.no_layers, batch_size, self.hidden_dim).requires_grad_()
        out, hidden = self.rnn_layer(x, hidden.detach())
        
        hidden = hidden.detach()
        out = out.view(-1, self.hidden_dim)

        out = self.linear_out(out)
        out = self.softmax(out)

        # out = out.view(batch_size, len(s))
        return out, hidden

In [73]:
input_size = len(s)
hidden_size = len(s) # 128
output_size = len(s)
rnn = RNN(input_size, hidden_size, output_size)

In [74]:
epochs = 100
lr = 1e-3
# Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr = lr)

## Training Run

In [75]:
out, hidden = rnn(X_train[0])
loss = criterion(out, Y_train[0])
# print(out[:2])
print(out.size())
print(loss)

torch.Size([25, 82])
tensor(4.3949, grad_fn=<NllLossBackward>)


In [76]:
print(len(Y_test[:2]))
len(Y_test)

2


1200

In [77]:
device = torch.device("cuda")

In [78]:
train_loss = []
val_error = []
counter = 0
rnn.train()

RNN(
  (rnn_layer): RNN(82, 82, batch_first=True)
  (linear_out): Linear(in_features=82, out_features=82, bias=True)
  (softmax): LogSoftmax(dim=1)
)

In [79]:
for epoch in range(1, epochs+1):
    for x, target in zip(X_train, Y_train):
        rnn.train()
        rnn.zero_grad()
        optimizer.zero_grad()
        x.to(device)
        target.to(device)
        output, hidden = rnn(x)
               
        loss = criterion(output, target)
        
        loss.backward() 
        nn.utils.clip_grad_norm_(rnn.parameters(), 1)
        optimizer.step() 

        
        if(counter % 1000) == 0:
          if(counter % 1000000) == 0:
            print("Iteration: {}, Loss: {}".format(counter, loss))
          rnn.eval()
          train_loss.append(loss.item())
          correct = 0
          total = 0
          with torch.no_grad():
            rnn.eval()
            for x_test, y_test in zip(X_test[:10], Y_test[:10]):
              out, hidden = rnn(x_test)
              total += batch_size

              val = criterion(out, y_test)
              val_error.append(val)

              _, pred = torch.max(out.data, 1)
              correct += (pred == y_test).sum()
        counter += 1


    if epoch%5 == 0:
        print('Epoch: {}/{}.............'.format(epoch, epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

Iteration: 0, Loss: 4.394851207733154
Epoch: 5/100............. Loss: 3.1552
Epoch: 10/100............. Loss: 3.1551
Epoch: 15/100............. Loss: 3.1551
Epoch: 20/100............. Loss: 3.1551
Epoch: 25/100............. Loss: 3.1551
Iteration: 1000000, Loss: 4.099247932434082
Epoch: 30/100............. Loss: 3.1551
Epoch: 35/100............. Loss: 3.1551
Epoch: 40/100............. Loss: 3.1550
Epoch: 45/100............. Loss: 3.1550
Epoch: 50/100............. Loss: 3.1550
Iteration: 2000000, Loss: 4.099268913269043
Epoch: 55/100............. Loss: 3.1550


KeyboardInterrupt: ignored

In [ ]:
from matplotlib import pyplot as plt
import pandas as pd

In [ ]:
plt.plot(np.arange(len(val_error)), val_error)

In [ ]:
plt.plot(np.arange(len(train_loss)), train_loss)

In [ ]:
train_losses_df = pd.DataFrame(train_loss)
train_losses_df.to_csv('losses.csv')

In [ ]:
val_losses_df = pd.DataFrame(val_error)
val_losses_df.to_csv('val_error.csv')